# Train a Hypergraph Message Passing Neural Network (HMPNN)

In this notebook, we will create and train a Hypergraph Message Passing Neural Network in the hypergraph domain. This method is introduced in the paper [Message Passing Neural Networks for
Hypergraphs](https://arxiv.org/abs/2203.16995) by Heydari et Livi 2022. We will use a benchmark dataset, Cora, a collection of 2708 academic papers and 5429 citation relations, to do the task of node classification. There are 7 category labels, namely `Case_Based`, `Genetic_Algorithms`, `Neural_Networks`, `Probabilistic_Methods`, `Reinforcement_Learning`, `Rule_Learning` and `Theory`.

Each document is initially represented as a binary vector of length 1433, standing for a unique subset of the words within the papers, in which a value of 1 means the presence of its corresponding word in the paper.

In [1]:
import torch
from torch_geometric.datasets import Planetoid
from sklearn.metrics import accuracy_score

from topomodelx.nn.hypergraph.hmpnn import HMPNN

If GPU's are available, we will make use of them. Otherwise, this will run on CPU.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Pre-processing

Here we download the dataset. It contains initial representation of nodes, the adjacency information, category labels and train-val-test masks.

In [3]:
dataset = Planetoid(".", "cora")[0]

Processing...
Done!


Below, we construct the incidence matrix ($B_1$) which is of shape $n_\text{nodes} \times n_\text{edges}$.

In [4]:
dataset["incidence_1"] = torch.sparse_coo_tensor(
    dataset["edge_index"], torch.ones(dataset["edge_index"].shape[1]), dtype=torch.long
)
dataset = dataset.to(device)

# Train the Neural Network

We then specify the hyperparameters and construct the model, the loss and optimizer.

In [5]:
torch.manual_seed(41)

in_features = 1433
hidden_features = 8
num_classes = 7
n_layers = 2

model = HMPNN(in_features, (256, hidden_features), num_classes, n_layers).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

Now it's time to train the model, looping over the network for a low amount of epochs. We keep training minimal for the purpose of rapid testing.

In [6]:
train_y_true = dataset["y"][dataset["train_mask"]]
val_y_true = dataset["y"][dataset["val_mask"]]
initial_x_1 = torch.zeros_like(dataset["x"])
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    y_pred_logits = model(dataset["x"], initial_x_1, dataset["incidence_1"])
    loss = loss_fn(y_pred_logits[dataset["train_mask"]], train_y_true)
    loss.backward()
    optimizer.step()

    train_loss = loss.item()
    y_pred = y_pred_logits.argmax(dim=-1)
    train_acc = accuracy_score(train_y_true, y_pred[dataset["train_mask"]])

    model.eval()
    with torch.no_grad():
        y_pred_logits = model(dataset["x"], initial_x_1, dataset["incidence_1"])
    val_loss = loss_fn(y_pred_logits[dataset["val_mask"]], val_y_true).item()
    y_pred = y_pred_logits.argmax(dim=-1)
    val_acc = accuracy_score(val_y_true, y_pred[dataset["val_mask"]])
    print(
        f"Epoch: {epoch + 1} train loss: {train_loss:.4f} train acc: {train_acc:.2f} "
        f"val loss: {val_loss:.4f} val acc: {val_acc:.2f}"
    )

Epoch: 1 train loss: 2.1079 train acc: 0.14 val loss: 2.1436 val acc: 0.16
Epoch: 2 train loss: 2.0234 train acc: 0.15 val loss: 2.1016 val acc: 0.16
Epoch: 3 train loss: 1.9800 train acc: 0.15 val loss: 2.0681 val acc: 0.16
Epoch: 4 train loss: 1.9504 train acc: 0.18 val loss: 2.0389 val acc: 0.16
Epoch: 5 train loss: 1.9194 train acc: 0.21 val loss: 2.0137 val acc: 0.16
Epoch: 6 train loss: 1.9241 train acc: 0.19 val loss: 1.9917 val acc: 0.16
Epoch: 7 train loss: 1.8917 train acc: 0.19 val loss: 1.9729 val acc: 0.16
Epoch: 8 train loss: 1.8710 train acc: 0.23 val loss: 1.9556 val acc: 0.16
Epoch: 9 train loss: 1.8574 train acc: 0.29 val loss: 1.9402 val acc: 0.17
Epoch: 10 train loss: 1.8646 train acc: 0.29 val loss: 1.9265 val acc: 0.17
Epoch: 11 train loss: 1.8540 train acc: 0.33 val loss: 1.9137 val acc: 0.19
Epoch: 12 train loss: 1.8430 train acc: 0.36 val loss: 1.9012 val acc: 0.22
Epoch: 13 train loss: 1.8336 train acc: 0.38 val loss: 1.8886 val acc: 0.25
Epoch: 14 train loss:

Finally, we evaluate the model against the test data.

In [7]:
test_y_true = dataset["y"][dataset["test_mask"]]
initial_x_1 = torch.zeros_like(dataset["x"])
model.eval()
with torch.no_grad():
    y_pred_logits = model(dataset["x"], initial_x_1, dataset["incidence_1"])
test_loss = loss_fn(y_pred_logits[dataset["test_mask"]], test_y_true).item()
y_pred = y_pred_logits.argmax(dim=-1)
test_acc = accuracy_score(test_y_true, y_pred[dataset["test_mask"]])
print(f"Test loss: {test_loss:.4f} test acc: {test_acc:.2f} ")

Test loss: 1.3152 test acc: 0.55 
